In [1]:
import pandas as pd
import wandb
import torch
import os
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
from torchvision import transforms
import torch
# load own code
import sys
sys.path.append('../')
from sleeplib.Resnet_15.model import FineTuning
from sleeplib.datasets import BonoboDataset , ContinousToSnippetDataset
from sleeplib.montages import CDAC_bipolar_montage,CDAC_common_average_montage,CDAC_combine_montage
from sleeplib.transforms import cut_and_jitter, CDAC_bipolar_signal_flip,CDAC_monopolar_signal_flip
# this holds all the configuration parameters
from sleeplib.config import Config
import pickle


In [2]:
# define model name and path
model_path = 'Models/YOUR_MODEL_NAME'
# load config and show all default parameters
config = Config()
config.print_config()
# the config can be changed by setting config.PARAMETER = VALUE

# save config to model_path using pickle
with open(os.path.join(model_path, 'config.pkl'), 'wb') as f:
    pickle.dump(config, f)

THIS CONFIG FILE CONTAINS THE FOLLOWING PARAMETERS :

PATH_FILES_BONOBO /home/ubuntu/data/bonobo/npy/
PATH_LUT_BONOBO /home/ubuntu/code/Spike_transformer/lut_labelled_20230628.csv
PATH_CONTINOUS_EEG /bdsp/staging/Bonobo/Datasets/continuousEEG
FQ 128
MONTAGE bipolar
WINDOWSIZE 1
N_FFT 128
HOP_LENGTH 64
HEAD_DROPOUT 0.3
EMB_SIZE 256
HEADS 8
DEPTH 4
N_CHANNELS 37
BATCH_SIZE 128
LR 0.0001


In [3]:
# load dataset
df = pd.read_csv(config.PATH_LUT_BONOBO,sep=';')
# add transformations
transform_train_bipolar = transforms.Compose([cut_and_jitter(windowsize=config.WINDOWSIZE,max_offset=0.1,Fq=config.FQ), CDAC_bipolar_signal_flip(p=1)])
transform_train_common_average = transforms.Compose([cut_and_jitter(windowsize=config.WINDOWSIZE,max_offset=0.1,Fq=config.FQ), CDAC_monopolar_signal_flip(p=1)])
transform_val = transforms.Compose([cut_and_jitter(windowsize=config.WINDOWSIZE,max_offset=0,Fq=config.FQ)])#,CDAC_signal_flip(p=0)])
transform_val = transforms.Compose([cut_and_jitter(windowsize=config.WINDOWSIZE,max_offset=0,Fq=config.FQ)])#,CDAC_signal_flip(p=0)])

# init datasets
sub_df = df[df['total_votes_received']>2]
train_df = sub_df[sub_df['Mode']=='Train']
val_df = sub_df[sub_df['Mode']=='Val']

# set up dataloaders
bipolar_montage = CDAC_bipolar_montage()
common_average_montage = CDAC_common_average_montage()
combine_montage = CDAC_combine_montage()

Bonobo_train = BonoboDataset(train_df, config.PATH_FILES_BONOBO, 
                             transform_train_bipolar=transform_train_bipolar,
                             transform_train_common_average=transform_train_common_average, 
                             bipolar_montage=bipolar_montage, 
                             common_average_montage=common_average_montage,
                             combine_montage = combine_montage
                            )
train_dataloader = DataLoader(Bonobo_train, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=os.cpu_count())

Bonobo_val = BonoboDataset(val_df, config.PATH_FILES_BONOBO, 
                           transform_train_bipolar=transform_val, 
                           transform_train_common_average=transform_train_common_average, 
                           bipolar_montage=bipolar_montage, 
                           common_average_montage=common_average_montage,
                           combine_montage = combine_montage
                          )
val_dataloader = DataLoader(Bonobo_val, batch_size=config.BATCH_SIZE,shuffle=False,num_workers=os.cpu_count())

#Bonobo_con = ContinousToSnippetDataset('/home/ubuntu/data/Bonobo01742_0.mat',montage=montage)#montage use numpy,so .mat maybe cannot do it
#con_dataloader = DataLoader(Bonobo_con, batch_size=config.BATCH_SIZE,shuffle=False,num_workers=os.cpu_count())


In [4]:
# build model
model = FineTuning(lr=config.LR,
                    head_dropout=config.HEAD_DROPOUT,
                    n_channels=config.N_CHANNELS,
                    n_fft=config.N_FFT,
                    hop_length=config.HOP_LENGTH)

# create a logger
wandb.init(dir='logging')
wandb_logger = WandbLogger(project='super_awesome_project') 

# create callbacks with early stopping and model checkpoint (saves the best model)
callbacks = [EarlyStopping(monitor='val_loss',patience=5),ModelCheckpoint(dirpath=model_path,filename='weights',monitor='val_loss')]
# create trainer, use fast dev run to test the code
trainer = pl.Trainer(devices=1, accelerator="gpu", min_epochs=20,logger=wandb_logger,callbacks=callbacks,fast_dev_run=False)
# train the model
trainer.fit(model,train_dataloader,val_dataloader)
wandb.finish()

wandb: Currently logged in as: 540452385 (nickljlee). Use `wandb login --relogin` to force relogin
wandb: WARNING Path logging/wandb/ wasn't writable, using system temp directory.
wandb: WARNING Path logging/wandb/ wasn't writable, using system temp directory


/opt/conda/envs/eeg/lib/python3.8/site-packages/pytorch_lightning/loggers/wandb.py:396: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/envs/eeg/lib/python3.8/site-packages/pytorch_lightning/loops/utilities.py:70: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
/opt/conda/envs/eeg/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:615: UserWarning: Checkpoint directory /home/ubuntu/code/Spike_transformer/Models/YOUR_MODEL_NAME exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RAN

Sanity Checking: 0it [00:00, ?it/s]

Traceback (most recent call last):
  File "/opt/conda/envs/eeg/lib/python3.8/multiprocessing/util.py", line 300, in _run_finalizers
    finalizer()
  File "/opt/conda/envs/eeg/lib/python3.8/multiprocessing/util.py", line 224, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/opt/conda/envs/eeg/lib/python3.8/multiprocessing/util.py", line 133, in _remove_temp_dir
    rmtree(tempdir)
  File "/opt/conda/envs/eeg/lib/python3.8/shutil.py", line 722, in rmtree
    onerror(os.rmdir, path, sys.exc_info())
  File "/opt/conda/envs/eeg/lib/python3.8/shutil.py", line 720, in rmtree
    os.rmdir(path)
OSError: [Errno 39] Directory not empty: '/tmp/pymp-jmd8j5qs'
Traceback (most recent call last):
  File "/opt/conda/envs/eeg/lib/python3.8/multiprocessing/util.py", line 300, in _run_finalizers
    finalizer()
  File "/opt/conda/envs/eeg/lib/python3.8/multiprocessing/util.py", line 224, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/opt/conda/envs/e

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/opt/conda/envs/eeg/lib/python3.8/site-packages/pytorch_lightning/trainer/call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


epoch,▁▃▄▄▄▄▄▅▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇██
train_loss_epoch,█▆▄▃▃▃▂▂▂▂▁▁▁
train_loss_step,▇▇▃▃▃▂▃▄▆▆█▅▅▅▂▅▁▃▄▁▃▃▄▃▆▄▅▄▃▅▂▅▃▄▁▃▄▂▃▄
trainer/global_step,▁▃▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇████
val_loss,▁▃▇██▇██▇▇▇█▇
epoch,13
train_loss_epoch,0.05628
train_loss_step,0.0583
trainer/global_step,1527
val_loss,0.07205


In [ ]:
'''
import scipy.io as sio
import numpy as np
import h5py
# load config and show all default parameters
config = Config()
config.print_config()
df = pd.read_csv(config.PATH_LUT_BONOBO,sep=';')


for idx in range(0,32433):
    event_file = df.iloc[idx]['event_file']
    # load signal of the idx-th sample
    mat_file_name = f"{event_file}.mat"
    path_signal = os.path.join('/home/ubuntu/data/bonobo/1.5/',mat_file_name)
    signal = h5py.File(path_signal,'r')
    data = signal['seg']
    npy_file_name = f"{event_file}.npy"
    npy_file_path = os.path.join('/home/ubuntu/data/bonobo/npy1.5/', npy_file_name)
    # Save the matrix as .npy file
    np.save(npy_file_path, data)





In [5]:
for x, y in train_dataloader:
    with torch.no_grad():
        print(x.shape)

torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
torch.Size([128, 37, 128])
t